In [527]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score ,f1_score
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier,StackingClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer

pd.options.display.max_rows = 10

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re
import nltk as nlp

from sklearn.pipeline import make_pipeline

## loading

In [528]:
df = pd.read_csv('data_for_test.csv')

In [529]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   name                1448 non-null   object
 1   real_name           1301 non-null   object
 2   full_name           956 non-null    object
 3   overall_score       1450 non-null   object
 4   history_text        1360 non-null   object
 5   intelligence_score  1450 non-null   int64 
 6   strength_score      1450 non-null   int64 
 7   speed_score         1450 non-null   int64 
 8   durability_score    1450 non-null   int64 
 9   power_score         1450 non-null   int64 
 10  combat_score        1450 non-null   int64 
 11  creator             1311 non-null   object
 12  gender              1305 non-null   object
 13  type_race           1043 non-null   object
 14  height              1448 non-null   object
 15  weight              1448 non-null   object
dtypes: int64(6), object(10)


In [530]:
df

,name,real_name,full_name,overall_score,history_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,creator,gender,type_race,height,weight
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",85,30,60,60,40,70,Marvel Comics,Male,Human,-,-
1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,100,20,30,50,35,100,DC Comics,NaN,NaN,-,-
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...",80,100,80,100,100,80,Marvel Comics,Male,Human,6'8 • 203 cm,980 lb • 441 kg
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,80,50,55,45,100,55,DC Comics,Male,Human,-,-
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,80,10,25,40,30,50,DC Comics,Male,Human,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,90,10,25,30,100,55,DC Comics,Female,Human,5'7 • 170 cm,127 lb • 57 kg
1446,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,80,100,100,100,100,80,Capcom,Male,Robot,5'6 • 168 cm,145 lb • 65 kg
1447,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",95,50,100,75,100,80,DC Comics,Male,NaN,6'1 • 185 cm,181 lb • 81 kg
1448,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,75,10,100,30,100,30,DC Comics,Male,NaN,6'1 • 185 cm,181 lb • 81 kg


In [531]:
df.drop(columns=["name","real_name","full_name","height","weight"],inplace=True)
df.dropna(subset=['creator'],inplace=True)
df.dropna(subset=['history_text'],inplace=True)

In [532]:
df.overall_score = df.overall_score.replace(["∞","-"],"0")
df.overall_score = df.overall_score.astype(int)
df.overall_score = df.overall_score.replace(0,df.overall_score.mean())

In [533]:
for col in df.columns:
    print(f"{col} :  {df[col].isna().sum()/df.shape[0]*100}")

overall_score :  0.0
history_text :  0.0
intelligence_score :  0.0
strength_score :  0.0
speed_score :  0.0
durability_score :  0.0
power_score :  0.0
combat_score :  0.0
creator :  0.0
gender :  10.05677210056772
type_race :  29.034874290348746


In [194]:
ls = list(df.columns)
ls.remove('history_text')

for col in ls:
    print(f"{col} :  {df[col].unique()}")

overall_score :  [  6.          10.          20.          12.           5.
   8.           9.          22.          13.          29.
  12.57664234  19.           7.           2.          15.
  14.          34.          35.           4.          37.
   3.          70.          21.          11.         222.
   1.          32.          16.          18.          80.
  17.          28.          42.          71.          84.
  73.          23.         112.         229.          24.
  61.         120.          40.         142.         124.
 211.          27.          54.          33.          57.
 169.          30.         226.          77.         237.
  26.          38.          47.          46.          87.
  25.          60.          82.          78.          66.
  62.          31.          64.          50.          58.
  65.         108.         125.          41.          75.
  53.         152.          83.         146.          49.        ]
intelligence_score :  [ 85 100  80  90  95  75

In [534]:
df.gender = df.gender.fillna("Male")
df.gender = df.gender.map({"Male":1,"Female":2})

In [535]:
labelencoder = LabelEncoder()
df.creator = labelencoder.fit_transform(df.creator)

labelencoder = LabelEncoder()
df.type_race = labelencoder.fit_transform(df.type_race.astype(str))

# df.type_race = df.type_race.replace(62,np.nan)

In [536]:
df.drop(['creator',"history_text"],axis=1).isna().sum()

overall_score         0
intelligence_score    0
strength_score        0
speed_score           0
durability_score      0
power_score           0
combat_score          0
gender                0
type_race             0
dtype: int64

In [134]:
imputer = KNNImputer(n_neighbors=3)
df[["overall_score","intelligence_score","strength_score","speed_score",'durability_score',"power_score",'combat_score',"gender","type_race"]] = imputer.fit_transform(df.drop(['creator',"history_text"],axis=1))

In [537]:
df = pd.concat([df,pd.get_dummies(df.type_race, prefix='type_race')],axis=1)
df.drop(columns=["type_race"],inplace=True)

In [538]:
scaler = StandardScaler()

df[["overall_score","intelligence_score","strength_score","speed_score","durability_score","power_score","combat_score"]] = scaler.fit_transform(df[["overall_score","intelligence_score","strength_score","speed_score","durability_score","power_score","combat_score"]])

## Splitting 

In [539]:
train_x, test_x, train_y, test_y = train_test_split(df.drop(['history_text',"creator"],axis=1),df["creator"],random_state = 0)

In [540]:
def evaluation(model):
    
    model.fit(train_x, train_y)
    y_pred = model.predict(test_x)
    acc = accuracy_score(test_y, y_pred)
    
    print("Accuracy: {:.2f}".format(acc*100))
    
    f1 = f1_score(test_y, y_pred, average='macro')
    print('F1-Score: %.3f' % f1)

##  train without Text

In [542]:
SVM = SVC(max_iter=10000,C=10,gamma='auto',random_state=0)
evaluation(SVM)

Accuracy: 63.11
F1-Score: 0.081


In [544]:
LSVM = LinearSVC(max_iter=10000,C=10,random_state=0)
evaluation(LSVM)

Accuracy: 61.17
F1-Score: 0.130


C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [142]:
RFC = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=100, random_state=0))
evaluation(RFC)

Accuracy: 63.75
F1-Score: 0.176


In [143]:
GBC = make_pipeline(StandardScaler(), GradientBoostingClassifier(learning_rate=0.2,
    max_depth=5,
    n_estimators=200))
evaluation(GBC)

Accuracy: 57.61
F1-Score: 0.142


In [144]:
KNN = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5))
evaluation(KNN)

Accuracy: 59.55
F1-Score: 0.130


In [145]:
LG = make_pipeline(StandardScaler(), LogisticRegression(max_iter=5000))
evaluation(LG)

Accuracy: 59.87
F1-Score: 0.117


In [146]:
model = StackingClassifier([('SVM', SVM),
                            ('RFC', RFC),
                            ('GBC', GBC),
                           ('KNN', KNN)],
                             final_estimator=DecisionTreeClassifier())

evaluation(model)

C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:901: RuntimeWarning: Number of classes in training fold (33) does not match total number of classes (36). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '
C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:901: RuntimeWarning: Number of classes in training fold (34) does not match total number of classes (36). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training f

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [378]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

model = BaggingClassifier(base_estimator=SVC(),
                         n_estimators=100)

model.fit(train.drop(['creator'],axis=1), train["creator"])

BaggingClassifier(base_estimator=SVC(), n_estimators=100)

## Splitting

In [451]:
train_x, test_x,train_y,test_y = train_test_split(df['history_text'],df['creator'],test_size = 0.2,random_state = 0)

In [452]:
def eval_text(model):
    model.fit(train_x, train_y)
    y_pred = model.predict(test_x)
    acc = accuracy_score(test_y, y_pred)
    print("Accuracy: {:.2f}".format(acc*100))
    f1 = f1_score(test_y, y_pred, average='macro')
    print('F1-Score: %.2f' % (f1*100))

##  train withText

In [450]:
i = 0

def preprocess_text(text):
    chars="\n,."
    for char in chars:
        text = text.replace(char," ")
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    lst_text = text.split()
    lst_stopwords = stopwords.words("french")
    lst_text = [word for word in lst_text if word not in lst_stopwords]
    ps = PorterStemmer()
    lst_text = [ps.stem(word) for word in lst_text]
    lem = WordNetLemmatizer()
    lst_text = [lem.lemmatize(word) for word in lst_text]
    ############
    global i
    i+=1
    print(f'{i} / {df.shape[0]}')
    ###############
    text = " ".join(lst_text)
    return text

df["history_text"] = df["history_text"].apply(preprocess_text)  

1 / 1233
2 / 1233
3 / 1233
4 / 1233
5 / 1233
6 / 1233
7 / 1233
8 / 1233
9 / 1233
10 / 1233
11 / 1233
12 / 1233
13 / 1233
14 / 1233
15 / 1233
16 / 1233
17 / 1233
18 / 1233
19 / 1233
20 / 1233
21 / 1233
22 / 1233
23 / 1233
24 / 1233
25 / 1233
26 / 1233
27 / 1233
28 / 1233
29 / 1233
30 / 1233
31 / 1233
32 / 1233
33 / 1233
34 / 1233
35 / 1233
36 / 1233
37 / 1233
38 / 1233
39 / 1233
40 / 1233
41 / 1233
42 / 1233
43 / 1233
44 / 1233
45 / 1233
46 / 1233
47 / 1233
48 / 1233
49 / 1233
50 / 1233
51 / 1233
52 / 1233
53 / 1233
54 / 1233
55 / 1233
56 / 1233
57 / 1233
58 / 1233
59 / 1233
60 / 1233
61 / 1233
62 / 1233
63 / 1233
64 / 1233
65 / 1233
66 / 1233
67 / 1233
68 / 1233
69 / 1233
70 / 1233
71 / 1233
72 / 1233
73 / 1233
74 / 1233
75 / 1233
76 / 1233
77 / 1233
78 / 1233
79 / 1233
80 / 1233
81 / 1233
82 / 1233
83 / 1233
84 / 1233
85 / 1233
86 / 1233
87 / 1233
88 / 1233
89 / 1233
90 / 1233
91 / 1233
92 / 1233
93 / 1233
94 / 1233
95 / 1233
96 / 1233
97 / 1233
98 / 1233
99 / 1233
100 / 1233
101 / 12

KeyboardInterrupt: 

In [453]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(df['history_text'])

TfidfVectorizer()

In [454]:
train_x = vectorizer.transform(train_x)
test_x = vectorizer.transform(test_x)

In [455]:
ETC = ExtraTreesClassifier(n_estimators=200)
ETC.fit(train_x, train_y)

predicted = ETC.predict(test_x)

acc = accuracy_score(test_y, predicted)
print("Accuracy: {:.2f}".format(acc*100))

Accuracy: 76.52


In [213]:
SVM = SVC(random_state=0)
eval_text(SVM)

Accuracy: 76.92
F1-Score: 13.082


In [215]:
LSVM = LinearSVC(random_state=0)
eval_text(SVM)

Accuracy: 76.92
F1-Score: 13.08


In [216]:
RFC = RandomForestClassifier(max_depth=100, random_state=0)
eval_text(RFC)

Accuracy: 69.64
F1-Score: 7.02


In [225]:
GBC = GradientBoostingClassifier(learning_rate=0.2,
    max_depth=5,
    n_estimators=200)
eval_text(GBC)

Accuracy: 75.71
F1-Score: 39.45


In [217]:
ABC = AdaBoostClassifier(n_estimators=100)
eval_text(ABC)

Accuracy: 56.68
F1-Score: 14.72


In [224]:
KNN = KNeighborsClassifier(n_neighbors=3)
eval_text(KNN)

Accuracy: 74.90
F1-Score: 30.97


In [182]:
model = StackingClassifier([('SVM', SVM),
                            ('LSVM', LSVM),
                            ('RFC', RFC),
                            ('KNN', KNN),
                            ('ABC', ABC)],
                             final_estimator=KNeighborsClassifier())

eval_text(model)

C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:901: RuntimeWarning: Number of classes in training fold (35) does not match total number of classes (37). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '
C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:901: RuntimeWarning: Number of classes in trainin

Accuracy: 33.60
F1-Score: 0.020


In [183]:
model = BaggingClassifier(base_estimator=ABC,
                         n_estimators=10)

eval_text(model)

Accuracy: 55.47
F1-Score: 0.039


In [456]:
params = {
    'n_neighbors':range(1,20),
}

grid = GridSearchCV(KNN, param_grid=params, cv=4)

eval_text(grid)

C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"


Accuracy: 85.02
F1-Score: 34.22


In [185]:
grid.best_params_

{'n_neighbors': 1}

##  with both

In [265]:
y_pred = model.predict(test_x)

acc = accuracy_score(test_y, y_pred)
f1 = 
print("Accuracy : {:.2f}".format(acc*100))

Accuracy : 85.83


on va essayer de concatener les features extrait du history text avec les caractéristique de chaque héro pour voir si ça donne de meilleur résultats

In [129]:
tr = np.concatenate((train1.toarray(), train.drop(['history_text','creator'],axis=1).to_numpy()), axis=1)
ts = np.concatenate((test1.toarray(), test.drop(['history_text','creator'],axis=1).to_numpy()), axis=1)

In [130]:
model = LinearSVC()
model.fit(tr, train["creator"])

C:\Users\dada\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [131]:
y_pred = model.predict(ts)

acc = accuracy_score(test["creator"], y_pred)

print("Accuracy : {:.2f}".format(acc*100))

Accuracy : 67.00


# W2v


In [506]:
import nltk
from gensim.models import Word2Vec
from IPython.display import clear_output

In [507]:
i = 0

def preprocess_text(text):
    chars="\n,."
    for char in chars:
        text = text.replace(char," ")
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    lst_text = text.split()
    lst_stopwords = stopwords.words("french")
    lst_text = [word for word in lst_text if word not in lst_stopwords]
    ps = PorterStemmer()
    lst_text = [ps.stem(word) for word in lst_text]
    lem = WordNetLemmatizer()
    lst_text = [lem.lemmatize(word) for word in lst_text]
    ############
    global i
    i+=1
    clear_output(wait=True)
    print(f'{i} / {df.shape[0]}')
    ###############
    text = " ".join(lst_text)
    return text

df["clean_text"] = df["history_text"].apply(preprocess_text)  

1233 / 1233


In [508]:
train_x, test_x,train_y,test_y = train_test_split(df['clean_text'],df['creator'],test_size = 0.2,random_state = 0)

In [509]:
doc = df['clean_text'].tolist()

doc = " ".join(doc)

doc = doc.lower()
doc = re.sub('[^a-zA-Z]', ' ', doc )
doc = re.sub(r'\s+', ' ', doc)

all_sentences = nltk.sent_tokenize(doc)

all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

all_words = all_words[0]

model = Word2Vec(all_words, window=5, min_count=2, workers=2)

w2v = {w: vec for w, vec in zip(model.wv.index_to_key, model.wv.vectors)}

In [510]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim= model.vector_size
        else:
            self.dim=0
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [512]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier


etree_w2v = make_pipeline( MeanEmbeddingVectorizer(w2v), 
                         ExtraTreesClassifier(n_estimators=200))

etree_w2v.fit(train_x, train_y)
predicted = etree_w2v.predict(test_x)

acc = accuracy_score(test_y, predicted)
print("Accuracy: {:.2f}".format(acc*100))

Accuracy: 56.68


In [514]:
from sklearn.naive_bayes import GaussianNB
clf = make_pipeline( MeanEmbeddingVectorizer(w2v), 
                        GaussianNB())
clf.fit(train_x, train_y)

predicted = clf.predict(test_x)

acc = accuracy_score(test_y, predicted)
print("Accuracy: {:.2f}".format(acc*100))

Accuracy: 20.65


In [517]:
ETC =  make_pipeline( MeanEmbeddingVectorizer(w2v),RandomForestClassifier(n_estimators=200))
ETC.fit(x_train, train_y)

predicted = ETC.predict(x_test)

acc = accuracy_score(test_y, predicted)
print("Accuracy: {:.2f}".format(acc*100))

Accuracy: 48.99


In [525]:
params = {
    'kneighborsclassifier__n_neighbors':range(1,20),
}
KNN = make_pipeline( MeanEmbeddingVectorizer(w2v),KNeighborsClassifier(n_neighbors=3))
grid = GridSearchCV(KNN, param_grid=params, cv=4)

In [526]:
grid.fit(train_x, train_y)

predicted = grid.predict(test_x)

acc = accuracy_score(test_y, predicted)
print("Accuracy: {:.2f}".format(acc*100))

C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"


Accuracy: 51.42


In [552]:
x = pd.read_csv("data_for_test.csv")

In [553]:
x.dropna(subset=['creator'],inplace=True)
x.dropna(subset=['history_text'],inplace=True)

In [554]:
x = x[['history_text','creator']]

In [555]:
train, test = train_test_split(x,test_size = 0.2,random_state = 0)

In [557]:
train.to_csv("train.csv",index=False)
test.to_csv("test.csv",index=False)